In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from collections import OrderedDict

from rewire import *

In [ ]:
def coolkids(self_state, neighs_infected, neighs_noninfected, global_infected):
    diff = neighs_infected - neighs_noninfected
    return diff
def evangelical(self_state, neighs_infected, neighs_noninfected, global_infected):
    # missionary - simplest
    return self_state*global_infected
    # in addition to i-i to i-n switching, also switching from n-n to n-i -- why?
    # wannabe proszelysers?
def veggie(self_state, neighs_infected, neighs_noninfected, global_infected):
    diff = neighs_infected - neighs_noninfected
    return self_state * diff

beta=100
rdictN = {}
for ndx, N in enumerate(10**np.linspace(2,3,10)):
    graph_density = 5.0 / N #  0.01
    mx_init = init_graph(N, graph_density)
    #print 'init  q', graphy.louvain.optimize_modularity(mx_init)[1]
    is_infected_init = (np.random.random(N) < .1)
    #rdicts_db = {}
    #rdicts_db['ER'] = rdicts
    
    rdictN[ndx] = run_rewire(mx_init, is_infected_init, benefit_function=veggie, 
                                       opts=dict(beta=beta, p_transmit=0.001, NUM_ITERS=1000))
    print ndx, N, rdictN[ndx]['num_infected'][-1],
    ni=np.array(rdictN[ndx]['num_infected'],dtype='float')
    s=150
    i_over_doI = (ni[s:]-ni[:-s])/ni[s/2:-s/2]
    print i_over_doI.max()
#del mx_init, is_infected_init
    



In [ ]:
n=[]
plt.figure(figsize=(8,5))
plt.style.use('seaborn-white')

for p,r in rdictN.items():
    ni=np.array(r['num_infected'],dtype='float')
    #print ni
    s=150
    i_over_doI = (ni[s:]-ni[:-s])/ni[s/2:-s/2]
    n.append((ni.max(), i_over_doI.max()))
    plt.figure()
    plt.plot(i_over_doI)
    print ni.max(), i_over_doI.max()
plt.figure()
x,y=zip(*n) 
x=np.array(x,dtype='float')
y=np.array(y,dtype='float')
print x,y
plt.loglog( x, y , '*')
#N=2500.0
from scipy.stats import linregress
#linregress(x,y)
lr=linregress(np.log(x),np.log(y)) #x and y are arrays or lists.
print 'slope=',lr.slope
print 
plt.loglog( x, (x*np.exp(lr.intercept))**lr.slope, '--') # x^.78